# Benchmark data processing

Contains the code required to plot the figures in the docs.

First run the benchmarks with the build-bench Docker container and the CI python scripts, then copy the resulting benchmark-results.txt file out of the Docker container and run this notebook.

In [ ]:
! docker cp builder-bench:/root/boost-root/__build/benchmark-results.txt /tmp/benchmark-results.txt

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv('benchmarks2.txt', header=None)
df = df.set_index(df.index.map(lambda x: x % df.groupby(0).size()[0]))
df = df.pivot(columns=[0], values=[1])
df.columns = df.columns.get_level_values(0)
df.head()

In [ ]:
fig, _ = plt.subplots(1, 3, figsize=(20, 10))
df[['one_row_boost', 'one_row_libmysqlclient', 'one_row_libmariadb']].rename(columns={
    'one_row_boost': 'Boost.MySQL',
    'one_row_libmysqlclient': 'libmysqlclient',
    'one_row_libmariadb': 'libmariadb'
}).plot.box(ylim=(800, 1100), ax=fig.axes[0], title='Reading one row', ylabel='Time (ms)')
df[['many_rows_boost', 'many_rows_libmysqlclient', 'many_rows_libmariadb']].rename(columns={
    'many_rows_boost': 'Boost.MySQL',
    'many_rows_libmysqlclient': 'libmysqlclient',
    'many_rows_libmariadb': 'libmariadb'
}).plot.box(ylim=(2200, 2500), ax=fig.axes[1], title='Reading 10k rows')
df[['stmt_params_boost', 'stmt_params_libmysqlclient', 'stmt_params_libmariadb']].rename(columns={
    'stmt_params_boost': 'Boost.MySQL',
    'stmt_params_libmysqlclient': 'libmysqlclient',
    'stmt_params_libmariadb': 'libmariadb'
}).plot.box(ylim=(10350, 10650), ax=fig.axes[2], title='Statement with params');